[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/drive/1grgaSEJNPjs9G_zQMChc1IIO5rEDNwGR?usp=sharing)



# Install and import packages 


In [1]:
import os
import pandas as pd
import numpy as np

In [2]:
!pip install sentence_transformers
!pip install scikit-learn
!pip install unzip

     |████████████████████████████████| 79 kB 3.9 MB/s 
     |████████████████████████████████| 4.0 MB 10.5 MB/s 
     |████████████████████████████████| 1.2 MB 37.0 MB/s 
     |████████████████████████████████| 77 kB 727 kB/s 
     |████████████████████████████████| 895 kB 36.7 MB/s 
     |████████████████████████████████| 6.5 MB 33.9 MB/s 
     |████████████████████████████████| 596 kB 15.3 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.0-py3-none-any.whl size=120747 sha256=6ae8ec3c57630775a068cdad1a430f4db655eebb7cef91baa0ab5ccc8259ba33
  Stored in directory: /root/.cache/pip/wheels/83/c0/df/b6873ab7aac3f2465aa9144b6b4c41c4391cfecc027c8b07e7
Successfully built sentence-transformers
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
  Created wheel for unzip: filename=unzip-1.0.0-py3-none-any.whl size=1319 sha256=fac14da3b0b4a3d68466759d17e8ee7faf9225

In [3]:
from sentence_transformers import SentenceTransformer
from collections import Counter



In [4]:
#Upload csv.zip and text.zip from the drop box folder CogAI/synthea_colab
!unzip csv.zip
!unzip text.zip

Archive:  csv.zip
   creating: 2022_04_04T07_13_58Z/
  inflating: 2022_04_04T07_13_58Z/claims.csv  
  inflating: 2022_04_04T07_13_58Z/procedures.csv  
  inflating: 2022_04_04T07_13_58Z/allergies.csv  
  inflating: 2022_04_04T07_13_58Z/supplies.csv  
  inflating: 2022_04_04T07_13_58Z/devices.csv  
  inflating: 2022_04_04T07_13_58Z/encounters.csv  
  inflating: 2022_04_04T07_13_58Z/payers.csv  
  inflating: 2022_04_04T07_13_58Z/providers.csv  
  inflating: 2022_04_04T07_13_58Z/payer_transitions.csv  
  inflating: 2022_04_04T07_13_58Z/conditions.csv  
  inflating: 2022_04_04T07_13_58Z/claims_transactions.csv  
  inflating: 2022_04_04T07_13_58Z/patients.csv  
  inflating: 2022_04_04T07_13_58Z/organizations.csv  
  inflating: 2022_04_04T07_13_58Z/immunizations.csv  
  inflating: 2022_04_04T07_13_58Z/imaging_studies.csv  
  inflating: 2022_04_04T07_13_58Z/careplans.csv  
  inflating: 2022_04_04T07_13_58Z/medications.csv  
  inflating: 2022_04_04T07_13_58Z/observations.csv  
Archive:  text.zi

## Define Data location

In [8]:
csvs = {}
synthea =  os.getcwd()
csv_loc = synthea+'/2022_04_04T07_13_58Z/'
csv_names = os.listdir(csv_loc)
!mkdir encoded_text

# Load the model and encode any text

In [5]:
bioBERT = 'emilyalsentzer/Bio_ClinicalBERT'
model = SentenceTransformer(bioBERT)
example_embeddings = model.encode("we can embed anything")

Downloading:   0%|          | 0.00/391 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.07k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.67k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/385 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/9.61M [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/23.4k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/4.07M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/436M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/213k [00:00<?, ?B/s]

Some weights of the model checkpoint at /root/.cache/torch/sentence_transformers/emilyalsentzer_Bio_ClinicalBERT were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


# Text to embeded csv

In [9]:
from datetime import datetime


def text_to_csv(text_loc, text_name, time=False):
    start = datetime.now()
    patient = text_name[:-4]
    csv_name = patient+'.csv'
    csv_loc = os.path.abspath(text_loc+'../encoded_text/'+csv_name)
    if os.path.exists(os.path.abspath(csv_loc)):   #not (csv_name in os.listdir(os.path.abspath(text_loc+'../encoded_text/'))):
        embeded_text= pd.read_csv(csv_loc)
        length = len(embeded_text)
    else:
        file = open(text_loc+ text_name, encoding="utf8")
        text=file.readlines()
        embeding = list(model.encode(text))
        length =len(text)
        column_patient = len(text)*[patient]
        embeded_text = pd.DataFrame({'Patient':column_patient, 'Text': text, 'Embeding': embeding}, index=np.arange(0,length, 1))
        embeded_text.to_csv(csv_loc)
    stop = datetime.now()
    if time:
        seconds = (stop-start)
        rate = seconds/length
        print("This file took: ", seconds)
        print('At a rate of ' + str(rate) +' per line')
    return embeded_text

In [14]:
!ls

2022_04_04T07_13_58Z  csv.zip  encoded_text  sample_data  text	text.zip


Since there is so many files it might get stuck 

In [15]:
from tqdm.auto import tqdm
embeded_dfs =[]
text_loc = synthea+'/text/'
#print(text_loc+'../encoded_text/')
for text_name in tqdm(os.listdir(text_loc)):
    embeded_dfs.append(text_to_csv(text_loc, text_name))

  0%|          | 0/1140 [00:00<?, ?it/s]

KeyboardInterrupt: ignored

# Format patient info 

## Create a dictonary of the medical records csvs and a list of patients 

In [ ]:
csvs = {}
synthea =  os.getcwd()
csv_loc = synthea+'/2022_04_04T07_13_58Z/'
csv_names = os.listdir(csv_loc)
for i, name in enumerate(csv_names):
    csv_names[i] = name[:-4]
    csvs[name[:-4]] = pd.read_csv(csv_loc+ name)


In [ ]:

info_keys = ['allergies', 'medications', 'conditions', 'careplans', 
             'observations', 'procedures', 'immunizations', 'imaging_studies']

In [ ]:
ids = np.array(csvs['procedures']['PATIENT'])

for info_key in info_keys:
    csv = csvs[info_key]
    if 'PATIENTID' in csv.keys():
        key = "PATIENTID"
    else:
        key = "PATIENT"
    uniq_ids = csv[key].unique()
    ids = np.concatenate((ids, np.array(uniq_ids)), axis=0)
ids = np.unique(ids)
print('Number of patients: ', ids.shape[0])

Number of patients:  1140


Next we will make a list filled with a dictonary for each patient (patients_info). Each dictonary will contain keys for the patient name, each record csv, and info. Info contains the full text of all the record csvs for this patient.

gt_patients_info is a list where each index corresponds to the same index in patients_info and contains the true insurance codes coressponding to each patinet.

gt_dict, Lastly gt_dict is a dictonary containing a key for each patient found in "ids" and all the insurance codes for the associated key/patients 



In [ ]:
patients_info = []
gt_patients_info = []
gt_dict = {}
for patient in ids:
    patient_dict = {}
    patient_dict['name'] = patient
    info_list = []
    gt_list = []
    for info_key in info_keys:
        csv = csvs[info_key]
        # print(csv.keys())
        # print(info_key)
        if 'PATIENTID' in csv.keys():
            key = "PATIENTID"
        else:
            key = "PATIENT"
        try:
            data =np.array(csv.groupby(by = [key]).get_group(patient)['DESCRIPTION'])
            gt = np.array(csv.groupby(by = [key]).get_group(patient)['CODE'])
            patient_dict[info_key] = data
            info_list  = np.concatenate((info_list, data))
            gt_list = np.concatenate((gt_list, gt))
        except:
            
            pass
    patient_dict['info'] = info_list
    patients_info.append(patient_dict)
    gt_patients_info.append(gt_list) 
    gt_dict[patient] = gt_list

In [ ]:
patients_info[0].keys()


dict_keys(['name', 'medications', 'conditions', 'careplans', 'observations', 'procedures', 'immunizations', 'info'])

In [ ]:
# If you run the code to process the .zips this will encode a single patients record
# patient_text_embed = model.encode(texts[0])
patient_procedures_embed = model.encode(patients_info[0]['procedures'])

NameError: name 'patients_info' is not defined

# Put text files into dictionary and into a list

Text_dict simply contains all of the patients record names as keys and the file text content as the value

In [ ]:
texts_dict = {}
texts = []
text_loc = synthea+'/text/text/'
text_names = os.listdir(text_loc)
for i, name in enumerate(text_names):
    text_names[i] = name[:-4]
    file = open(text_loc+ name, encoding="utf8")
    text=file.readlines()
    # with open(text_loc+ name) as f:
    #     text = f.readlines()
    texts_dict[name[:-4]] = text
    texts.append(text)